In [12]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from pathlib import Path

from src.scripts.esi_surrogate import forward
from src.scripts.geom_sampler import lhc_sampler, D_BOUNDS, RC_BOUNDS, ALPHA_BOUNDS, H_BOUNDS, RA_BOUNDS

data_dir = Path('../../data/geometry')

In [20]:
problem = {
    'num_vars': 5,
    'names': ['d', 'rc', 'alpha', 'h', 'ra'],
    'bounds': [[x*1e-6 for x in D_BOUNDS], [x*1e-6 for x in RC_BOUNDS], [x*np.pi/180 for x in ALPHA_BOUNDS],
               [x*1e-6 for x in H_BOUNDS] , [x*1e-6 for x in RA_BOUNDS]]
}

N = 7000
samples = lhc_sampler(N)
emax = forward(samples.T, net_file=str(data_dir/'models'/'esi_surrogate.onnx'),
                    data_file=str(data_dir/'models'/'norm_data.mat'))

In [21]:
Si = sobol.analyze(problem, emax, print_to_console=True, calc_second_order=False)

             ST   ST_conf
d      1.200368  0.234317
rc     1.089495  0.192769
alpha  1.019339  0.152323
h      1.057362  0.163800
ra     0.986757  0.157882
             S1   S1_conf
d      0.055325  0.095489
rc     0.030180  0.094446
alpha  0.024489  0.082700
h      0.038121  0.085495
ra     0.062585  0.084021


In [22]:
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-np.pi, np.pi]]*3,
}

# Sample
param_values = saltelli.sample(problem, 1024)

# Run model
Y = Ishigami.evaluate(param_values)

# Sobol sensitivity indices
Si = sobol.analyze(problem, Y, print_to_console=True)

          ST   ST_conf
x1  0.555860  0.077531
x2  0.441898  0.040259
x3  0.244675  0.027902
          S1   S1_conf
x1  0.316832  0.055590
x2  0.443763  0.049807
x3  0.012203  0.055550
                S2   S2_conf
(x1, x2)  0.009254  0.076015
(x1, x3)  0.238172  0.110699
(x2, x3) -0.004888  0.059443


In [23]:
problem

{'num_vars': 3,
 'names': ['x1', 'x2', 'x3'],
 'bounds': [[-3.141592653589793, 3.141592653589793],
  [-3.141592653589793, 3.141592653589793],
  [-3.141592653589793, 3.141592653589793]],
 'sample_scaled': True}

In [19]:
type(Y)

numpy.ndarray

In [10]:
Si

{'S1': array([0.31683154, 0.44376306, 0.01220312]),
 'S1_conf': array([0.06730461, 0.05665592, 0.0572674 ]),
 'ST': array([0.55586009, 0.44189807, 0.24467539]),
 'ST_conf': array([0.07614813, 0.03993115, 0.02402399]),
 'S2': array([[        nan,  0.00925429,  0.23817211],
        [        nan,         nan, -0.0048877 ],
        [        nan,         nan,         nan]]),
 'S2_conf': array([[       nan, 0.08673279, 0.10968803],
        [       nan,        nan, 0.06650353],
        [       nan,        nan,        nan]])}

In [ ]:
alpha = 30 * np.pi/180
rc = 30e-6
d = 500e-6
ra = 300e-6
N = 100
h = np.linspace(50e-6, 1000e-6, N)

x = np.vstack((np.ones((1,N))*d, np.ones((1,N))*rc, np.ones((1,N))*alpha, h[np.newaxis, :], np.ones((1,N))*ra))
emax = forward(x, net_file='../../data/geometry/models/esi_surrogate.onnx', data_file='../../data/geometry/models/norm_data.mat',
               V0=1000)
plt.figure()
plt.plot(h, emax)
plt.xlabel('Emitter height (um)')
plt.ylabel('Max electric field magnitude (V/m)')
plt.show()